# Business Problem:

## Background

## Problem Statement

Predict which schools have high and which schools have low high school graduation rates. Identify which characteristics are the best indicator of high and low graduation rates so that school districts know where to focus resources when attempting to increase high school graduation rates.

# Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
# import xgboost as xgb
import catboost
import time
import warnings
warnings.filterwarnings('ignore')

from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from catboost import Pool, sum_models
from catboost import CatBoostClassifier
from statsmodels.formula.api import ols
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, balanced_accuracy_score
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder,  OneHotEncoder, MinMaxScaler, StandardScaler 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform, truncnorm, randint


# Data Exploration

## Data Load

In [4]:
df_school = pd.read_csv('data/EducationDataPortal_schools.csv')
df_district = pd.read_csv('data/EducationDataPortal_districts.csv')

## Data Fields

In [5]:
df_school.head()

,year,ncessch,school_name,state_name,lea_name,zip_mailing,urban_centric_locale,school_level,school_type,charter,...,enrl_algebra2,enrl_physics,enrl_geometry,instances_mech_restraint,instances_phys_restraint,instances_seclusion,students_mech_restraint,students_phys_restraint,students_seclusion,students_SAT_ACT
0,2017,360000104498,JAMES A GREEN HIGH SCHOOL,New York,DOLGEVILLE CENTRAL SCHOOL DISTRICT,13329,"Rural, distant",High,Regular school,No,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017,360000204500,SAUQUOIT VALLEY HIGH SCHOOL,New York,SAUQUOIT VALLEY CENTRAL SCHOOL DISTRICT,13456,"Rural, fringe",High,Regular school,No,...,NaN,32.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,28.0
2,2017,360000304543,EDWARDS-KNOX JUNIOR-SENIOR HIGH SCHOOL,New York,EDWARDS-KNOX CENTRAL SCHOOL DISTRICT,13652,"Rural, remote",High,Regular school,No,...,NaN,7.0,31.0,0.0,0.0,0.0,0.0,0.0,0.0,26.0
3,2017,360000404545,MOHONASEN SENIOR HIGH SCHOOL,New York,ROTTERDAM-MOHONASEN CENTRAL SCHOOL DISTRICT,12303,"Suburb, large",High,Regular school,No,...,NaN,53.0,223.0,0.0,0.0,0.0,0.0,0.0,0.0,177.0
4,2017,360000500282,BROADALBIN-PERTH HIGH SCHOOL,New York,BROADALBIN-PERTH CENTRAL SCHOOL DISTRICT,12025,"Rural, distant",High,Regular school,No,...,NaN,41.0,101.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0


In [7]:
df_school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 76 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   year                             8188 non-null   int64  
 1   ncessch                          8188 non-null   int64  
 2   school_name                      8188 non-null   object 
 3   state_name                       8188 non-null   object 
 4   lea_name                         8188 non-null   object 
 5   zip_mailing                      8188 non-null   int64  
 6   urban_centric_locale             8188 non-null   object 
 7   school_level                     8188 non-null   object 
 8   school_type                      8188 non-null   object 
 9   charter                          8188 non-null   object 
 10  enrollment                       8188 non-null   object 
 11  read_test_num_valid              7942 non-null   float64
 12  read_test_pct_prof_l

- Combine grad rates into single binary column
- Drop outlay_capital_arra 
- nulls in all enrl fields
- nulls in corp inst

In [ ]:
def target_col (row):
   if row['eri_hispanic   '] == 1 :
      return 'Hispanic'
   if row['eri_afr_amer'] + row['eri_asian'] + row['eri_hawaiian'] + row['eri_nat_amer'] + row['eri_white'] > 1 :
      return 'Two Or More'
   if row['eri_nat_amer'] == 1 :
      return 'A/I AK Native'
   if row['eri_asian'] == 1:
      return 'Asian'
   if row['eri_afr_amer']  == 1:
      return 'Black/AA'
   if row['eri_hawaiian'] == 1:
      return 'Haw/Pac Isl.'
   if row['eri_white'] == 1:
      return 'White'
   return 'Other'


In [6]:
df_district.head()

,year,leaid,lea_name,state_name,city_location,agency_type,enrollment,rev_total,rev_fed_total,rev_fed_child_nutrition_act,...,payments_state_govt,payments_local_govt,payments_other_sch_system,salaries_total,salaries_instruction,benefits_employee_total,cohort_num,grad_rate_high,grad_rate_low,grad_rate_midpt
0,2017,3600001,DOLGEVILLE CENTRAL SCHOOL DISTRICT,New York,DOLGEVILLE,Regular local school district,929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,82.0,89,85,87
1,2017,3600002,SAUQUOIT VALLEY CENTRAL SCHOOL DISTRICT,New York,SAUQUOIT,Regular local school district,1033,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,88.0,94,90,92
2,2017,3600003,EDWARDS-KNOX CENTRAL SCHOOL DISTRICT,New York,HERMON,Regular local school district,550,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,48.0,89,80,84
3,2017,3600004,ROTTERDAM-MOHONASEN CENTRAL SCHOOL DISTRICT,New York,SCHENECTADY,Regular local school district,2843,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,239.0,88,88,88
4,2017,3600005,BROADALBIN-PERTH CENTRAL SCHOOL DISTRICT,New York,BROADALBIN,Regular local school district,1829,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,139.0,94,90,92


In [8]:
df_district.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8063 entries, 0 to 8062
Data columns (total 68 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   year                            8063 non-null   int64  
 1   leaid                           8063 non-null   int64  
 2   lea_name                        8063 non-null   object 
 3   state_name                      8063 non-null   object 
 4   city_location                   8063 non-null   object 
 5   agency_type                     8063 non-null   object 
 6   enrollment                      7991 non-null   object 
 7   rev_total                       7012 non-null   object 
 8   rev_fed_total                   7012 non-null   object 
 9   rev_fed_child_nutrition_act     7012 non-null   object 
 10  rev_fed_state_title_i           7012 non-null   object 
 11  rev_fed_state_math_sci_teach    7012 non-null   object 
 12  rev_fed_state_drug_free         70

In [9]:
df_district['enrollment'].value_counts(normalize=True,dropna=False)

Not applicable    0.025301
0                 0.014635
NaN               0.008930
304               0.001860
355               0.001612
                    ...   
3146              0.000124
2455              0.000124
4483              0.000124
29436             0.000124
20360             0.000124
Name: enrollment, Length: 3706, dtype: float64

In [ ]:
# Plot of the target status group

plt.hist(df_schools['grad_rate'])
plt.xlabel('grad_rate', fontsize=14)
plt.title("Distribution of Graduation Rate", fontsize=16)
plt.show()


# Data Preparation

## Missing Values

## Columns to drop

### Mostly Empty

### Many Individual Values

### Not Significant

## Categorical and Numerical

### Join Target: df_train_set to df_train

### Column Binning

# Train Test Split

# Model Development